# Exploring Halo5 API Results

- Adapted from Johnny Dryman's Code: https://github.com/Drymander/halo-upload-2

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns',0)

# Warnings
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
import ast
import time
import http.client, urllib.request, urllib.parse, urllib.error, base64
import pickle
from tqdm import tqdm
import isodate

# Pull Most Recent Match Details

This is a combination of two API calls - the first is to pull the latest match for a given player based on their gamertag.  This provides the Match ID, and the second API call pulls the data using the Match ID. 

In [3]:
import json
with open("/Users/jamesirving/.secret/halo-api.json") as f:
    api_login = json.load(f)
api_login.keys()

dict_keys(['primary-key', 'secondary-key'])

In [4]:
api_key = api_login['primary-key']

### Using Johnny's Code

- get results form player stats api to find the last match played. 
- Save the match ID from the player results and then use the match-endpoint to get match details. 

In [5]:
## Getting initial player stats with most recent match
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': api_key,
}

player = 'Drymander'

params = urllib.parse.urlencode({
    # Request parameters
    'modes': 'arena',
    'start': 0,
    'count': 1,
    'include-times': True,
})

conn = http.client.HTTPSConnection('www.haloapi.com')
conn.request("GET", f"/stats/h5/players/{player}/matches?%s" % params, "{body}", headers)
response = conn.getresponse()
latest_match = json.loads(response.read())
latest_match

{'Start': 0,
 'Count': 1,
 'ResultCount': 1,
 'Results': [{'Links': {'StatsMatchDetails': {'AuthorityId': 'spartanstats',
     'Path': 'h5/arena/matches/ca0c20ca-c780-45f4-bc61-a654b3564e12',
     'QueryString': None,
     'RetryPolicyId': 'exponentialretry',
     'TopicName': '',
     'AcknowledgementTypeId': 0,
     'AuthenticationLifetimeExtensionSupported': False,
     'ClearanceAware': False},
    'UgcFilmManifest': {'AuthorityId': 'ugc',
     'Path': '/h5/films/ca0c20ca-c780-45f4-bc61-a654b3564e12',
     'QueryString': '?view=film-manifest',
     'RetryPolicyId': 'exponentialretry',
     'TopicName': '',
     'AcknowledgementTypeId': 0,
     'AuthenticationLifetimeExtensionSupported': False,
     'ClearanceAware': False}},
   'Id': {'MatchId': 'ca0c20ca-c780-45f4-bc61-a654b3564e12', 'GameMode': 1},
   'HopperId': 'f0c9ef9a-48bd-4b24-9db3-2c76b4e23450',
   'MapId': 'd67fdcb9-6d9c-403e-960d-04202e19b244',
   'MapVariant': {'ResourceType': 3,
    'ResourceId': '8e2e0fbd-47d3-4ede-8f

## Now must find `MatchId` and Variant ID

### Approach 1: Manually exploring + pprint

In [6]:
from pprint import pprint
pprint(latest_match)#Results

{'Count': 1,
 'Links': {'Self': {'AcknowledgementTypeId': 0,
                    'AuthenticationLifetimeExtensionSupported': False,
                    'AuthorityId': 'spartanstats',
                    'ClearanceAware': False,
                    'Path': '/h5/players/Drymander/matches',
                    'QueryString': '?include-xuids=false&view=full&include-ratings=false&modes=arena&start=0&count=1&include-times=True',
                    'RetryPolicyId': '',
                    'TopicName': ''}},
 'ResultCount': 1,
 'Results': [{'GameBaseVariantId': 'a2949322-dc84-45ab-8454-cf94fb28c189',
              'GameVariant': {'Owner': '',
                              'OwnerType': 3,
                              'ResourceId': '0df797e5-6505-4c98-a5cd-16f9a06f8b25',
                              'ResourceType': 2},
              'HopperId': 'f0c9ef9a-48bd-4b24-9db3-2c76b4e23450',
              'Id': {'GameMode': 1,
                     'MatchId': 'ca0c20ca-c780-45f4-bc61-a654b3564e12'},
 

In [7]:
## Saving JSON results
results = latest_match['Results']
len(results)

1

In [8]:
result = results[0]
result.keys()

dict_keys(['Links', 'Id', 'HopperId', 'MapId', 'MapVariant', 'GameBaseVariantId', 'GameVariant', 'MatchDuration', 'MatchCompletedDate', 'Teams', 'Players', 'IsTeamGame', 'SeasonId', 'MatchCompletedDateFidelity'])

In [9]:
pprint(result)

{'GameBaseVariantId': 'a2949322-dc84-45ab-8454-cf94fb28c189',
 'GameVariant': {'Owner': '',
                 'OwnerType': 3,
                 'ResourceId': '0df797e5-6505-4c98-a5cd-16f9a06f8b25',
                 'ResourceType': 2},
 'HopperId': 'f0c9ef9a-48bd-4b24-9db3-2c76b4e23450',
 'Id': {'GameMode': 1, 'MatchId': 'ca0c20ca-c780-45f4-bc61-a654b3564e12'},
 'IsTeamGame': True,
 'Links': {'StatsMatchDetails': {'AcknowledgementTypeId': 0,
                                 'AuthenticationLifetimeExtensionSupported': False,
                                 'AuthorityId': 'spartanstats',
                                 'ClearanceAware': False,
                                 'Path': 'h5/arena/matches/ca0c20ca-c780-45f4-bc61-a654b3564e12',
                                 'QueryString': None,
                                 'RetryPolicyId': 'exponentialretry',
                                 'TopicName': ''},
           'UgcFilmManifest': {'AcknowledgementTypeId': 0,
                   

In [10]:
result['Id']['MatchId']

'ca0c20ca-c780-45f4-bc61-a654b3564e12'

In [11]:
match_id = latest_match['Results'][0]["Id"]["MatchId"]
match_id

'ca0c20ca-c780-45f4-bc61-a654b3564e12'

In [12]:
variant_id = latest_match['Results'][0]['GameBaseVariantId']
variant_id

'a2949322-dc84-45ab-8454-cf94fb28c189'

In [13]:
def get_match_info_from_player(latest_match):
    match_info = {'match_id':latest_match['Results'][0]["Id"]["MatchId"],
    'variant_id':latest_match['Results'][0]['GameBaseVariantId']}
    return match_info
    
    
ids = get_match_info_from_player(latest_match)
ids

{'match_id': 'ca0c20ca-c780-45f4-bc61-a654b3564e12',
 'variant_id': 'a2949322-dc84-45ab-8454-cf94fb28c189'}

#### Getting Match Stats with MatchID

In [14]:
def get_match_results_from_id(match_id):
    conn.request("GET", f"/stats/h5/arena/matches/{match_id}?%s" % params, "{body}", headers)
    response = conn.getresponse()

    match_results = json.loads(response.read())
    conn.close()
    return match_results

In [15]:
match_results= get_match_results_from_id(ids['match_id'])
match_results

{'PlayerStats': [{'XpInfo': {'PrevSpartanRank': 148,
    'SpartanRank': 148,
    'PrevTotalXP': 15015532,
    'TotalXP': 15016211,
    'SpartanRankMatchXPScalar': 2.248,
    'PlayerTimePerformanceXPAward': 277,
    'PerformanceXP': 0,
    'PlayerRankXPAward': 25,
    'BoostAmount': 0,
    'MatchSpeedWinAmount': 0,
    'ObjectivesCompletedAmount': 0},
   'PreviousCsr': None,
   'CurrentCsr': None,
   'MeasurementMatchesLeft': 10,
   'RewardSets': [],
   'KilledOpponentDetails': [{'GamerTag': 'SixKnifeWalker', 'TotalKills': 2},
    {'GamerTag': 'OoMSRoO FniX', 'TotalKills': 1},
    {'GamerTag': 'AuntieEcoDot', 'TotalKills': 3}],
   'KilledByOpponentDetails': [{'GamerTag': 'AuntieEcoDot', 'TotalKills': 4},
    {'GamerTag': 'CommandMedal52', 'TotalKills': 1},
    {'GamerTag': 'SixKnifeWalker', 'TotalKills': 3},
    {'GamerTag': 'OoMSRoO FniX', 'TotalKills': 2}],
   'FlexibleStats': {'MedalStatCounts': [],
    'ImpulseStatCounts': [{'Id': '7cda10c8-04c4-4916-8d6e-71a426a61de8',
      'Count

### Approach 2: Functionize exploring the keys of a dict

In [16]:
def dict_report(match_results):
    print("d=top-level dict:")
    print('==='*20)

    for key,values in match_results.items():

        print(f"\n- d['{key}']:")
        print("type:",
              type(values))

        ## if list: 
        if type(values)==list:

            # Print len, 
            print(f"len: {len(values)}")
            item = values[0]

            ## print first item
            print(f"\n\t- d['{key}'][0]:")

            ## if dict, print only keys
            if type(item)==dict:
                print(item.keys())
            else:
                print(item)

        elif type(values)==dict:
            print("\t",values.keys())

        else:
            print("\tvalue:",values)
            
dict_report(match_results)

d=top-level dict:

- d['PlayerStats']:
type: <class 'list'>
len: 8

	- d['PlayerStats'][0]:
dict_keys(['XpInfo', 'PreviousCsr', 'CurrentCsr', 'MeasurementMatchesLeft', 'RewardSets', 'KilledOpponentDetails', 'KilledByOpponentDetails', 'FlexibleStats', 'CreditsEarned', 'MetaCommendationDeltas', 'ProgressiveCommendationDeltas', 'BoostInfo', 'Player', 'TeamId', 'Rank', 'DNF', 'AvgLifeTimeOfPlayer', 'PreMatchRatings', 'PostMatchRatings', 'PlayerScore', 'GameEndStatus', 'TotalKills', 'TotalHeadshots', 'TotalWeaponDamage', 'TotalShotsFired', 'TotalShotsLanded', 'WeaponWithMostKills', 'TotalMeleeKills', 'TotalMeleeDamage', 'TotalAssassinations', 'TotalGroundPoundKills', 'TotalGroundPoundDamage', 'TotalShoulderBashKills', 'TotalShoulderBashDamage', 'TotalGrenadeDamage', 'TotalPowerWeaponKills', 'TotalPowerWeaponDamage', 'TotalPowerWeaponGrabs', 'TotalPowerWeaponPossessionTime', 'TotalDeaths', 'TotalAssists', 'TotalGamesCompleted', 'TotalGamesWon', 'TotalGamesLost', 'TotalGamesTied', 'TotalTimeP

In [17]:

def dict_dropdown(dict_to_display,title='Dictionary Contents'):
    """Display the model_params dictionary as a dropdown menu."""
    from ipywidgets import interact
    from IPython.display import display
    from pprint import pprint

    dash='---'
    print(f'{dash*4} {title} {dash*4}')

    @interact(dict_to_display=dict_to_display)
    def display_params(dict_to_display=dict_to_display):

        # # if the contents of the first level of keys is dicts:, display another dropdown
        # if dict_to_display.values()
        return pprint(dict_to_display)
         #params.values();
    
dict_dropdown(match_results)

------------ Dictionary Contents ------------


interactive(children=(Dropdown(description='dict_to_display', options={'PlayerStats': [{'XpInfo': {'PrevSparta…

In [18]:
# def unpack_dict_df(row,col='ProgressiveCommendationDeltas'):
#     for k,v in row[col].items():
#         row[f"{col}_{k}"] = v
#     return row

In [19]:
# df.apply(lambda x: unpack_dict_df())

In [21]:
# raise Exception('end of study group code')

# Build Match Dataframe

In [22]:
i = 0

df_match = pd.DataFrame()
columns = [
    'TeamId',
    'Gamertag',
    'SpartanRank',
    'PrevTotalXP',
]
df_match = pd.DataFrame(columns = columns)

for player in match_results['PlayerStats']:

    player_dic = {}
    # Team ID
    player_dic['TeamId'] = match_results['PlayerStats'][i]['TeamId']
    # Gamer Tag
    player_dic['Gamertag'] = match_results['PlayerStats'][i]['Player']['Gamertag']
    # Spartan Rank
    player_dic['SpartanRank'] = match_results['PlayerStats'][i]['XpInfo']['SpartanRank']
    # Previous Total XP
    player_dic['PrevTotalXP'] = match_results['PlayerStats'][i]['XpInfo']['PrevTotalXP']
    df_match = df_match.append(player_dic, ignore_index=True)
    i += 1
    
df_match

# playerteam = df_match.loc[df_match['Gamertag'] == user, 'TeamId'].values[0]

# playerteam

,TeamId,Gamertag,SpartanRank,PrevTotalXP
0,0,Drymander,148,15015532
1,0,Wittensgirl,44,634368
2,0,Xv PIPELAYER Xv,147,11889011
3,1,OoMSRoO FniX,152,72837898
4,1,AuntieEcoDot,149,20438952
5,0,RoOkIePhEnOm10,84,2439106
6,1,CommandMedal52,148,15704146
7,1,SixKnifeWalker,151,36708272


# Pull Game Base Variant Stats for All Players in Match

Using the match details, we will compile everyone's gamertags into a list and pull more specific information for each player.  We can get statistics on every game mode (or "Game Base Variant" as it is referred to in the API).

In [23]:
player_list = str(list(df_match['Gamertag']))[1:-1]

player_list
player_list = player_list.replace(', ',',')
player_list = player_list.replace("'",'')
player_list = player_list.replace(' ','+')

player_list

'Drymander,Wittensgirl,Xv+PIPELAYER+Xv,OoMSRoO+FniX,AuntieEcoDot,RoOkIePhEnOm10,CommandMedal52,SixKnifeWalker'

In [24]:

########### Python 3.2 #############
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': str(api_key),
}

params = urllib.parse.urlencode({
    # Request parameters
#     'seasonId': '{string}',
   
})

try:
    conn = http.client.HTTPSConnection('www.haloapi.com')
    conn.request("GET", f"/stats/h5/servicerecords/arena?players={player_list}&%s" % params, "{body}", headers)
    
    response = conn.getresponse()
    data = response.read()
    players_stats = json.loads(data)
#     print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

####################################
players_stats

{'Results': [{'Id': 'Drymander',
   'ResultCode': 0,
   'Result': {'ArenaStats': {'ArenaPlaylistStats': [{'PlaylistId': '7ec8ea9d-e672-4aa1-b4c3-5d0e4d0824c9',
       'MeasurementMatchesLeft': 9,
       'HighestCsr': None,
       'Csr': None,
       'CsrPercentile': None,
       'TotalKills': 9,
       'TotalHeadshots': 0,
       'TotalWeaponDamage': 1015.8301537036896,
       'TotalShotsFired': 253,
       'TotalShotsLanded': 101,
       'WeaponWithMostKills': {'WeaponId': {'StockId': 3168248199,
         'Attachments': []},
        'TotalShotsFired': 0,
        'TotalShotsLanded': 0,
        'TotalHeadshots': 0,
        'TotalKills': 4,
        'TotalDamageDealt': 512.5044994354248,
        'TotalPossessionTime': 'PT0S'},
       'TotalMeleeKills': 2,
       'TotalMeleeDamage': 417.5044994354248,
       'TotalAssassinations': 2,
       'TotalGroundPoundKills': 0,
       'TotalGroundPoundDamage': 0.0,
       'TotalShoulderBashKills': 0,
       'TotalShoulderBashDamage': 95.0,
       'T

## Compile Individual Stats into Dataframe

In [25]:
variant_id = match_results['GameBaseVariantId']
variant_id

'a2949322-dc84-45ab-8454-cf94fb28c189'

In [26]:
vdf_columns = [
    'Gamertag',
    'TotalTimePlayed',
    'K/D',
    'Accuracy',
    'WinRate',
]

vdf = pd.DataFrame(columns = vdf_columns)

i = 0

for player in players_stats['Results']:

    
    for variant in player['Result']['ArenaStats']['ArenaGameBaseVariantStats']:
        if variant['GameBaseVariantId'] == variant_id:
            variant_stats = variant
#         else:
#             continue
            
    variant_dic = {}
    variant_dic['Gamertag'] = players_stats['Results'][i]['Id']
    variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
    variant_dic['K/D'] = variant_stats['TotalKills'] / variant_stats['TotalDeaths']
    variant_dic['Accuracy'] = variant_stats['TotalShotsLanded'] / variant_stats['TotalShotsFired']
    variant_dic['WinRate'] = variant_stats['TotalGamesWon'] / variant_stats['TotalGamesLost']
    vdf = vdf.append(variant_dic, True)
    
    i += 1
    
vdf

,Gamertag,TotalTimePlayed,K/D,Accuracy,WinRate
0,Drymander,278.007420,0.875475,0.357355,1.341969
1,Wittensgirl,15.806765,0.736708,0.347182,0.887324
2,Xv PIPELAYER Xv,112.897528,0.900785,0.360279,0.820961
3,OoMSRoO FniX,311.117283,1.229809,0.385706,1.641290
4,AuntieEcoDot,141.889302,0.864368,0.399830,1.120000
5,RoOkIePhEnOm10,60.670997,0.772650,0.305874,1.190871
6,CommandMedal52,130.488255,1.090894,0.331354,0.852941
7,SixKnifeWalker,188.490374,0.877007,0.342756,0.970784


# Merge Match and Game Variant Dataframes

In [27]:
df_stats = pd.merge(df_match, vdf, how='inner', on = 'Gamertag')

df_stats.round(2)

,TeamId,Gamertag,SpartanRank,PrevTotalXP,TotalTimePlayed,K/D,Accuracy,WinRate
0,0,Drymander,148,15015532,278.01,0.88,0.36,1.34
1,0,Wittensgirl,44,634368,15.81,0.74,0.35,0.89
2,0,Xv PIPELAYER Xv,147,11889011,112.90,0.90,0.36,0.82
3,1,OoMSRoO FniX,152,72837898,311.12,1.23,0.39,1.64
4,1,AuntieEcoDot,149,20438952,141.89,0.86,0.40,1.12
5,0,RoOkIePhEnOm10,84,2439106,60.67,0.77,0.31,1.19
6,1,CommandMedal52,148,15704146,130.49,1.09,0.33,0.85
7,1,SixKnifeWalker,151,36708272,188.49,0.88,0.34,0.97


# Unused Code

In [ ]:
for variant in players_stats['Results'][0]['Result']['ArenaStats']['ArenaGameBaseVariantStats']:
    if variant['GameBaseVariantId'] == variant_id:
        variant_stats = variant
    else:
        continue
        
variant_stats['TotalKills']

### All stats

In [ ]:
# pip install isodate

import isodate

variant_dic = {}
variant_dic['Gamertag'] = players_stats['Results'][0]['Id']
variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
variant_dic['K/D'] = variant_stats['TotalKills'] / variant_stats['TotalDeaths']
variant_dic['Accuracy'] = variant_stats['TotalShotsLanded'] / variant_stats['TotalShotsFired']
variant_dic['WinRate'] = variant_stats['TotalGamesWon'] / variant_stats['TotalGamesLost']

variant_dic

In [ ]:
vdf.append(variant_dic, ignore_index=True)

In [ ]:
# variant_dic['TotalHeadshots']= variant_stats['TotalHeadshots']
# variant_dic['TotalWeaponDamage']= variant_stats['TotalWeaponDamage']
# variant_dic['TotalShotsFired']= variant_stats['TotalShotsFired']
# variant_dic['TotalShotsLanded']= variant_stats['TotalShotsLanded']
# variant_dic['TotalMeleeKills']= variant_stats['TotalMeleeKills']
# variant_dic['TotalMeleeDamage']= variant_stats['TotalMeleeDamage']
# variant_dic['TotalAssassinations']= variant_stats['TotalAssassinations']
# variant_dic['TotalGroundPoundKills']= variant_stats['TotalGroundPoundKills']
# variant_dic['TotalGroundPoundDamage']= variant_stats['TotalGroundPoundDamage']
# variant_dic['TotalShoulderBashKills']= variant_stats['TotalShoulderBashKills']
# variant_dic['TotalShoulderBashDamage']= variant_stats['TotalShoulderBashDamage']
# variant_dic['TotalGrenadeDamage']= variant_stats['TotalGrenadeDamage']
# variant_dic['TotalPowerWeaponKills']= variant_stats['TotalPowerWeaponKills']
# variant_dic['TotalPowerWeaponDamage']= variant_stats['TotalPowerWeaponDamage']
# variant_dic['TotalPowerWeaponGrabs']= variant_stats['TotalPowerWeaponGrabs']
# variant_dic['TotalPowerWeaponPossessionTime']= isodate.parse_duration(variant_stats['TotalPowerWeaponPossessionTime']).total_seconds() / 3600
# variant_dic['TotalDeaths']= variant_stats['TotalDeaths']
# variant_dic['TotalAssists']= variant_stats['TotalAssists']
# variant_dic['TotalGamesCompleted']= variant_stats['TotalGamesCompleted']
# variant_dic['TotalGamesWon']= variant_stats['TotalGamesWon']
# variant_dic['TotalGamesLost']= variant_stats['TotalGamesLost']
# variant_dic['TotalGamesTied']= variant_stats['TotalGamesTied']
# variant_dic['TotalGrenadeKills']= variant_stats['TotalGrenadeKills']
# variant_dic['TotalKills']= variant_stats['TotalKills']